<a href="https://colab.research.google.com/github/avshashov/solbridge_bot/blob/main/Konstantin_Teaching_Data_BBA_Clean_for_Each_Semester_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instructions and Steps:**
1.  For each new semester, click on Runtime from the menu and select "Restart and run all". Otherwise data might overlap from previous semester.
2. Upload the file from the 5th floor to the Google folder
3. Navigate to the folder in the left panel, right click the needed file, and copy the path. 
4. Paste that path in the field "sourceFile"
5. The output file "Refined File" will be in the folder "Files from 5th floor". Copy the data from "Refined File" to the main file. Delete the data in "Refined file" so that it is empty to run the script again next semester.

In [ ]:
# Use these to upload a local file instead

#from google.colab import files
#uploaded = files.upload()
#import pandas as pd
#filename = next(iter(uploaded))
#df1 = pd.ExcelFile(filename)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Path of the source file (raw file from the 5th floor)
# Navigate to the folder in the left panel and copy the path
sourceFile = "/content/drive/MyDrive/SolBridge/Administrative/Faculty Management/Data/Teaching Data/Files from 5th Floor/BBA/Course Evaluation Result_Winter 2022.xls" #@param {type:"string"}

In [ ]:
pip install --upgrade pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install --upgrade gspread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# For authentication
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
# To enable interactive tables
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Creating two master lists of semesters and years
semester_List = ['Spring', 'Summer', 'Fall', 'Winter']
year_List = list(range(2010,2101))

# Create a list of frames to store the data
listOfFrames = []

In [ ]:
df1 = pd.ExcelFile(sourceFile)     

In [ ]:
# Pull the first cell (first row, first column) of the raw file to get Year and Semester
yr_sem_raw = df1.parse(0, usecols=[0]).columns.values.tolist()

for year in year_List:
  if str(year) in str(yr_sem_raw):
    yr = year
for semester in semester_List:
  if semester in str(yr_sem_raw):
    sem = semester

# Joining Year and Semester
yr_sem = str(yr) + " " + sem

# Select only the relevant columns (keep "Type of Lecture" because I can use it later to filter out values)
df2 = df1.parse(0, usecols=[3,5,6,9,11,12,14,15,17,18], names = ['ID','Name','Position', 'Course Name', 'Section', 'Type of Lecture', 'Registered', 'Responded', '% Responded','Score'])

# Remove the first few rows that don't have data
df2 = df2[4:]

# For each row starting with the one containing a professor's name, if the row below is empty (which will be because of the merge), copy the value of the top row into the bottom row 
df2[['ID','Name','Position','Section','Score']] = df2[['ID','Name','Position','Section','Score']].ffill()

# In Spring and Summer 2017, the course types were General or Common. In recent times, it seems everything is "Block (day) "
df2 = df2.replace("General ", "Block (day) ")
df2 = df2.replace("Common ", "Block (day) ")
# Use this filter to remove unnecessary rows
df2 = df2[df2['Type of Lecture'] == "Block (day) "]

# Renaming the column 'Type of Lecture' and setting the level to 'BBA'                                  # For BBA, set the program as "BBA"
df2.rename(columns = {'Type of Lecture': 'Program'}, inplace= True)
df2 = df2.replace("Block (day) ", "BBA")

# Adding the information about year and semester
df2["Yr_Sem"] = yr_sem

# Changing column types
df2[['ID', 'Registered', 'Responded']] = df2[['ID', 'Registered', 'Responded']].astype(int)
df2[['% Responded', 'Score']] = df2[['% Responded', 'Score']].astype(float)
# Adding the refined data to the list of frames
listOfFrames.append(df2)

In [ ]:
# For some weird reason, the output (listOfFrames) had the filenames in it because of which latter steps (e.g., concat) would not run.
# Therefore, creating a new list to remove all other objects which are not dataframes 
from pandas.core.frame import DataFrame
newList = [value for value in listOfFrames if type(value) == DataFrame]

In [ ]:
# Combining all the dataframes into a single (master) dataframe
combinedDF = pd.concat(newList)

# Removing NAs
combinedDF = combinedDF.fillna("")

In [ ]:
# Apparently, the whole thing needs to be converted into lists before writing
combinedData = [combinedDF.columns.to_list()] + combinedDF.to_numpy().tolist()

In [ ]:
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# Enter the file ID and Sheet Name where the data is to be written (in this case the ID of the file called "Refined File" in my Google Drive)
wsMaster = gc.open_by_key("1JGdpWwDPeeVsuts2hKBY5lrcVG8pgw9XxflIELpl7Gs").worksheet("BBA")

In [ ]:
wsMaster.update("A3", combinedData, value_input_option="USER_ENTERED")

{'spreadsheetId': '1JGdpWwDPeeVsuts2hKBY5lrcVG8pgw9XxflIELpl7Gs',
 'updatedRange': 'BBA!A3:K29',
 'updatedRows': 27,
 'updatedColumns': 11,
 'updatedCells': 297}

In [ ]:
combinedDF

,ID,Name,Position,Course Name,Section,Program,Registered,Responded,% Responded,Score,Yr_Sem
5,20121186,길미현,Adjunct Professor,Japanese Intermediate 2,001,BBA,1,1,100.00,5.00,2022 Winter
7,20121186,길미현,Adjunct Professor,Japanese Advanced 2,001,BBA,3,2,66.67,5.00,2022 Winter
9,20211050,Paul Thomas Johnson,Full-time Faculty,Critical Thinking,001,BBA,10,9,90.00,4.85,2022 Winter
11,20101262,Takeda Yuko,Full-time Faculty,Japanese Intermediate 2,001,BBA,1,1,100.00,5.00,2022 Winter
13,20101262,Takeda Yuko,Full-time Faculty,Japanese Beginner 2,001,BBA,5,4,80.00,4.90,2022 Winter
15,20101591,Miller Jeffrey Alan,Full-time Faculty,Writing and Presentation Skills,001,BBA,22,12,54.55,4.34,2022 Winter
17,20191252,Annemari Ferreira,Full-time Faculty,Specialization Project - Marketing Capstone D...,001,BBA,2,1,50.00,5.00,2022 Winter
19,20201263,Wei He,Full-time Faculty,Chinese Intermediate 2,001,BBA,3,3,100.00,4.95,2022 Winter
21,20201368,Mengyuan Xu,Full-time Faculty,Chinese Advanced 2,001,BBA,4,2,50.00,5.00,2022 Winter
23,20211066,Shoko Yamane,Full-time Faculty,Japanese Advanced 2,001,BBA,3,2,66.67,5.00,2022 Winter
